# Processing Big Data - Deequ Analysis

© Explore Data Science Academy

## Honour Code
I {**ONIDAJO**, **WALE-OLAITAN**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.


## Context

Having completed manual data quality checks, it should be obvious that the process can become quite cumbersome. As the Data Engineer in the team, you have researched some tools that could potentially save the team from having to do this cumbersome work. In your research, you have come a across a tool called [Deequ](https://github.com/awslabs/deequ), which is a library for measuring the data quality of large datasets.

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://github.com/Explore-AI/Pictures/raw/master/data_engineering/transform/predict/DataQuality.jpg"
     alt="Data Quality"
     style="float: center; padding-bottom=0.5em"
     width=100%/>
     <p><em>Figure 1. Six dimensions of data quality</em></p>
</div>

You present this tool to your manager; he is quite impressed and gives you the go-ahead to use this in your implementation. You are now required to perform some data quality tests using this automated data testing tool.
 

> ## 🚩️ Important Notice 🚩️
>
>To successfully run `pydeequ` without any errors, please make sure that you have an environment that is running pyspark version 3.0.
> You are advised to **create a new conda environment** and install this specific version of pyspark to avoid any technical issues:
>
> `pip install pyspark==3.0`

<br>

## Import dependencies

If you do not have `pydeequ` already installed, install it using the following command:
- `pip install pydeequ`

In [5]:
pip install pydeequ

Note: you may need to restart the kernel to use updated packages.


In [2]:
import findspark
findspark.init()

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydeequ
from pydeequ.analyzers import *
from pydeequ.profiles import *
from pydeequ.suggestions import *
from pydeequ.checks import *
from pydeequ.verification import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType, DoubleType, IntegerType, DateType, NumericType, StructType, StringType, StructField

c:\Users\Asus\anaconda3\envs\pyspark_envo\lib\site-packages\ipykernel\parentpoller.py:110: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  warnings.warn(
Please set env variable SPARK_VERSION


In [2]:
spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

## Read data into spark dataframe

In this notebook, we set out to run some data quality tests, with the possiblity of running end to end on the years 1963, 1974, 1985, 1996, 2007, and 2018. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Data_ingestion_student_version.ipynb` notebook to create the parquet files for the following years:
>       - 1963
>       - 1974
>       - 1985
>       - 1996
>       - 2007
>       - 2018
>
>2. Ingest the data for the for the years given above. You should only do it one year at a time.
>3. Ingest the metadata file.


When developing your code, it will be sufficient to focus on a single year. However, after your development is done, you will need to run this notebook for all of the given years above so that you can answer all the questions given in the Data Testing MCQ.

In [3]:
#TODO: Write your code here
# Use this variable (year) to determine which year your are focusing on
year = 1963
df = spark.read.parquet('../sixty_three_transformed_data')

In [6]:
symbols_meta = spark.read.csv('../../../../Desktop/Explore/Processing Big Data/symbols_valid_meta.csv', header=True)

In [14]:
symbols_meta.show()

+-------------+------+--------------------+----------------+---------------+---+--------------+----------+----------------+----------+-------------+----------+
|Nasdaq Traded|Symbol|       Security Name|Listing Exchange|Market Category|ETF|Round Lot Size|Test Issue|Financial Status|CQS Symbol|NASDAQ Symbol|NextShares|
+-------------+------+--------------------+----------------+---------------+---+--------------+----------+----------------+----------+-------------+----------+
|            Y|     A|Agilent Technolog...|               N|               |  N|         100.0|         N|            null|         A|            A|         N|
|            Y|    AA|Alcoa Corporation...|               N|               |  N|         100.0|         N|            null|        AA|           AA|         N|
|            Y|  AAAU|Perth Mint Physic...|               P|               |  Y|         100.0|         N|            null|      AAAU|         AAAU|         N|
|            Y|  AACG|ATA Creativity Gl.

## **Run tests on the dataset**

## Test 1 - Null values ⛔️
For the first test, you are required to check the data for completeness.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for missing values in the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*


In [ ]:
d_data = ["date", "open","high","low","volume","close","adj_close", "volume"]

In [42]:
#TODO: Write your code here

check = Check(spark, CheckLevel.Warning, "Missing Values Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.areComplete(d_data)) \
    .run()
    
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+--------------------+-----------+------------+--------------------+-----------------+------------------+
|               check|check_level|check_status|          constraint|constraint_status|constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+------------------+
|Missing Values Check|    Warning|     Success|ComplianceConstra...|          Success|                  |
+--------------------+-----------+------------+--------------------+-----------------+------------------+



In [6]:
#TODO: Write your code here

check = Check(spark, CheckLevel.Warning, "Missing Values Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.isComplete("date") \
        .isComplete("open") \
        .isComplete("high") \
        .isComplete("low") \
        .isComplete("close") \
        .isComplete("adj_close") \
        .isComplete("volume")) \
    .run()
    
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+--------------------+-----------+------------+--------------------+-----------------+------------------+
|               check|check_level|check_status|          constraint|constraint_status|constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+------------------+
|Missing Values Check|    Warning|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|    Warning|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|    Warning|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|    Warning|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|    Warning|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|    Warning|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|    Warning|     Success

## Test 2 - Zero Values 🅾️

For the second test, you are required to check for zero values within the dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for zero values within the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [7]:
#TODO: Write your code here
check = Check(spark, CheckLevel.Warning, "Zeros Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasMin('open', lambda x: x == 1)
        .hasMin('high', lambda x: x == 1)  \
        .hasMin('low', lambda x: x == 1)  \
        .hasMin('close', lambda x: x == 1)  \
        .hasMin('adj_close', lambda x: x == 1)  \
        .hasMin('volume', lambda x: x == 1))  \
    .run()
    
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+-----------+-----------+------------+--------------------+-----------------+--------------------+
|      check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+-----------+-----------+------------+--------------------+-----------------+--------------------+
|Zeros Check|    Warning|     Warning|MinimumConstraint...|          Failure|Value: 0.0 does n...|
+-----------+-----------+------------+--------------------+-----------------+--------------------+



## Test 3 - Negative values ➖️
The third test requires you to check that all values in the data are positive.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check negative values within the dataset. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [7]:
#TODO: Write your code here

check = Check(spark, CheckLevel.Warning, "Negative value Review Check")
checkResult = VerificationSuite(spark) \
            .onData(df) \
            .addCheck(
            check.isNonNegative('open') \
            .isNonNegative('high') \
            .isNonNegative('low') \
            .isNonNegative('close') \
            .isNonNegative('adj_close') \
            .isNonNegative('volume')) \
            .run()
            
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+--------------------+-----------+------------+--------------------+-----------------+------------------+
|               check|check_level|check_status|          constraint|constraint_status|constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+------------------+
|Negative value Re...|    Warning|     Success|ComplianceConstra...|          Success|                  |
|Negative value Re...|    Warning|     Success|ComplianceConstra...|          Success|                  |
|Negative value Re...|    Warning|     Success|ComplianceConstra...|          Success|                  |
|Negative value Re...|    Warning|     Success|ComplianceConstra...|          Success|                  |
|Negative value Re...|    Warning|     Success|ComplianceConstra...|          Success|                  |
|Negative value Re...|    Warning|     Success|ComplianceConstra...|          Success|                  |
+--------------------+-----------+------------

## Test 4 - Determine Maximum Values ⚠️

For the fourth test, we want to find the maximum values in the dataset for the numerical fields. Extremum values can often be used to define an upper bound for the column values so we can define them as the threshold values. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Column Profiler Runner` to generate summary statistics for all the available columns. 
>2. Extract the maximum values for all the numeric columns in the data.
>
> *You may use as many cells as necessary*

In [21]:
#TODO: Write your code here

result = ColumnProfilerRunner(spark) \
    .onData(df) \
    .run()

for col, profile in result.profiles.items():
    print('profile:', profile)



profile: StandardProfiles for column: stock: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 20,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 0,
        "String": 5020
    },
    "histogram": [
        [
            "NAV",
            251,
            0.05
        ],
        [
            "MO",
            251,
            0.05
        ],
        [
            "XOM",
            251,
            0.05
        ],
        [
            "HPQ",
            251,
            0.05
        ],
        [
            "AA",
            251,
            0.05
        ],
        [
            "PG",
            251,
            0.05
        ],
        [
            "GT",
            251,
            0.05
        ],
        [
            "FL",
            251,
            0.05
        ],
        [
            "IP",
            251,
            0.05
        ],
        

In [19]:
max_values = {c: result.profiles[c].maximum if result.profiles[c].dataType == "Integral" or result.profiles[c].dataType == "Fractional" else None for c in df.columns}


In [20]:
max_values

{'date': None,
 'open': 303.125,
 'high': 315.625,
 'low': 311.875,
 'close': 313.75,
 'adj_close': 148.7704620361328,
 'volume': 20692800.0,
 'stock': None}

## Test 5 - Stock Tickers 💹️

For the fifth test, we want to determine if the stock tickers contained in our dataset are consistent. To do this, you will need to make use of use of the metadata file to check that the stock names used in the dataframe are valid. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine if the stock tickers contained in the dataset appear in the metadata file.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [36]:
symbol_list = [data[0] for data in symbols_meta.select('NASDAQ Symbol').collect()]

In [37]:
symbol_list

['A',
 'AA',
 'AAAU',
 'AACG',
 'AADR',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAU',
 'AAWW',
 'AAXJ',
 'AAXN',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABEO',
 'ABEQ',
 'ABEV',
 'ABG',
 'ABIO',
 'ABM',
 'ABMD',
 'ABR',
 'ABT',
 'ABTX',
 'ABUS',
 'AC',
 'ACA',
 'ACAD',
 'ACAM',
 'ACAMU',
 'ACB',
 'ACBI',
 'ACC',
 'ACCO',
 'ACEL',
 'ACER',
 'ACES',
 'ACGL',
 'ACGLO',
 'ACGLP',
 'ACH',
 'ACHC',
 'ACHV',
 'ACIA',
 'ACIO',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACOR',
 'ACP',
 'ACRE',
 'ACRS',
 'ACRX',
 'ACSG',
 'ACSI',
 'ACST',
 'ACT',
 'ACTG',
 'ACTT',
 'ACTTU',
 'ACU',
 'ACV',
 'ACWF',
 'ACWI',
 'ACWV',
 'ACWX',
 'ACY',
 'ADAP',
 'ADBE',
 'ADC',
 'ADES',
 'ADI',
 'ADIL',
 'ADM',
 'ADMA',
 'ADME',
 'ADMP',
 'ADMS',
 'ADNT',
 'ADP',
 'ADPT',
 'ADRE',
 'ADRO',
 'ADS',
 'ADSK',
 'ADSW',
 'ADT',
 'ADTN',
 'ADUS',
 'ADVM',
 'ADX',
 'ADXN',
 'ADXS',
 'AE',
 'AEB',
 'AEE',
 'AEF',
 'AEFC',
 'AEG',
 'AEGN',
 'AEHR',
 'AEIS',
 'AEL

In [39]:
#TODO: Write your code here
check = Check(spark, CheckLevel.Warning, "Consistency Review Check")
checkResult = VerificationSuite(spark) \
            .onData(df) \
            .addCheck(
            check
            .isContainedIn('stock', symbol_list)) \
            .run()
            
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+--------------------+-----------+------------+--------------------+-----------------+------------------+
|               check|check_level|check_status|          constraint|constraint_status|constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+------------------+
|Consistency Revie...|    Warning|     Success|ComplianceConstra...|          Success|                  |
+--------------------+-----------+------------+--------------------+-----------------+------------------+



## Test 6 - Duplication 👥️
Lastly, we want to determine the uniqueness of the items found in the dataframe. You need to make use of the Verification Suite to check for the validity of the stock tickers. 

Similar to the previous notebook - `Data_profiling_student_version.ipynb`, the first thing to check will be if the primary key values within the dataset are unique - in our case, that will be a combination of the stock name and the date. Secondly, we want to check if the entries are all unique, which is done by checking for duplicates across that whole dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine the uniqueness of entries contained within the dataset.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*



In [10]:
check = Check(spark, CheckLevel.Warning, "Duplication Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.isUnique("stock", "date"))  \
    .run()
    
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+-----------------+-----------+------------+--------------------+-----------------+--------------------+
|            check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+-----------------+-----------+------------+--------------------+-----------------+--------------------+
|Duplication Check|    Warning|     Warning|UniquenessConstra...|          Failure|Value: 0.0 does n...|
+-----------------+-----------+------------+--------------------+-----------------+--------------------+



In [11]:
#TODO: Write your code here
check = Check(spark, CheckLevel.Warning, "Duplication Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.isUnique('date')  \
        .isUnique('open')  \
        .isUnique('high')  \
        .isUnique('low')  \
        .isUnique('close')  \
        .isUnique('adj_close')  \
        .isUnique('volume')  \
        .isUnique('stock'))  \
    .run()
    
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+-----------------+-----------+------------+--------------------+-----------------+--------------------+
|            check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+-----------------+-----------+------------+--------------------+-----------------+--------------------+
|Duplication Check|    Warning|     Warning|UniquenessConstra...|          Failure|Value: 0.0 does n...|
|Duplication Check|    Warning|     Warning|UniquenessConstra...|          Failure|Value: 0.08127490...|
|Duplication Check|    Warning|     Warning|UniquenessConstra...|          Failure|Value: 0.12470119...|
|Duplication Check|    Warning|     Warning|UniquenessConstra...|          Failure|Value: 0.12390438...|
|Duplication Check|    Warning|     Warning|UniquenessConstra...|          Failure|Value: 0.12788844...|
|Duplication Check|    Warning|     Warning|UniquenessConstra...|          Failure|Value: 0.29541832...|
|Duplication Check|    Warning|     Warning|UniquenessC